In [5]:
from fortranformat import FortranRecordWriter as ffW
from pathlib import Path
import json

### Parser Class

In [6]:
class GeneralParser:
    config = {}

    def __init__(self,config):
        with open (Path(config)) as f:
            self.config=json.load(f)

    def __PrtToStr(self,name,bol):
        if bol:
            name += "=.true., "
        else:
            name += "=.false., "
        return name
    
    # Parse and print the NAMELIST line
    def __parPRT(self,configsub,data,filestream):
        filestream.write(" {} ".format(configsub["head"]))
        # Check keylist_A, all optional
        for var in configsub["keylist_A"]:
            if var in data:
                filestream.write(var.lower()+"="+data[var]+" ")
        
        # Check keylist_L, all optional
        for var in configsub["keylist_L"]:
            if var in data:
                filestream.write(self.__PrtToStr(var.lower(),data[var]))
        
        # Change Line, and write the "/" at the end of first line.
        filestream.write(" / \n")

    # Parse the variable line
    # All variable should have same type
    def __parVAL(self,configsub,data,filestream):
        countval = 0
        writer = ffW(configsub["format"])
        for var in configsub["keylist"]:
            if var in data:
                filestream.write(writer.write([data[var]]))
                countval+=1
            elif countval<configsub["minval"]:
                raise KeyError("Mandatory variable not provided: {}".format(var))
            else:
                break

        filestream.write("\n")

    # Print title
    def __parTITLE(self,configsub,data,filestream):
        if len(data["TITLE"]) > configsub["length"]:
            print("Warning: Title longer than config set length, will be capped")
        filestream.write(data["TITLE"][:configsub["length"]]+"\n")

    # Customised format
    def __parCUS(self,configsub,data,filestream):
        countval = 0
        for var in configsub["keylist"]:
            if var in data:
                writer = ffW(configsub["keylist"][var])
                filestream.write(writer.write([data[var]]))
                countval+=1
            elif countval<configsub["minval"]:
                raise KeyError("Mandatory variable not provided: {}".format(var))
            else:
                break

        filestream.write("\n")


    def write(self,input,output):
        with open (Path(output),"w+",encoding="utf-8") as f:
            for line in self.config:
                try:
                    if self.config[line]["type"] == "PRT":
                        self.__parPRT(self.config[line],input,f)
                    elif self.config[line]["type"] == "VAL":
                        self.__parVAL(self.config[line],input,f)
                    elif self.config[line]["type"] == "TITLE":
                        self.__parTITLE(self.config[line],input,f)
                    elif self.config[line]["type"] == "CUS":
                        self.__parCUS(self.config[line],input,f)
                    else:
                        raise ValueError("Config type not found: {}".format(line))
                except Exception as e:
                    print("Error parsing {}: {}".format(line,e))
                    raise
        



### Parser Testing

In [7]:
rot_example={
    "TOLER":"1.0d-4",
    "ZPFUN":True,
    "ZTRAN":True,
    "ZDCORE": False,
    "NVIB":40, "NEVAL":30,"KMIN":2,"IBASS":120,"NEVAL2":20,
    "TITLE":"HCN rovib J=2 e and f",
    "EZERO":3481.50552084
}

rot_example_missing={
    "TOLER":"1.0d-4",
    "ZPFUN":True,
    "ZTRAN":True,
    "ZDCORE": False,
            "NEVAL":30,"KMIN":2,"IBASS":120,"NEVAL2":20,
    "TITLE":"HCN rovib J=2 e and f",
    "EZERO":3481.50552084
}

In [8]:
testparser = GeneralParser("../configs/example_config.json")
testparser.write(rot_example,"../output/example.job")

---

In [20]:
dvrparalist={
    "ZROT":True,"ZTRAN":True,"ZLIN":True,"ZTHETA":False,"ZR2R1":False,"ZP1D":False,"ZP2D":False,"ZEMBED":False,"ZPFUN":True,
    "NCOORD":1,
    "NPNT2":30,"JROT":0,"NEVAL":40,"NALF":45,"MAX2D":1000,"MAX3D":1500,"IDIA":1,"KMIN":2,"NPNT1":16,
    "TITLE":"HCN: J=2 calculation",
    "XMASS(1)":1.007825035,"XMASS(2)":12.0,"XMASS(3)":14.003074002,
    "XMASSR(1)":1.007825035,"XMASSR(2)":12.0,"XMASSR(3)":14.003074002,
    "EMAX1":45000,
    "RE1":2.3,"DISS1":29.0,"WE1":0.0105,
    "RE2":3.2,"DISS2":5.0,"WE2":0.004
}

In [22]:
testparser = GeneralParser("configs/dvr3djz.json")
testparser.write(dvrparalist,"output/dvr_t.job")